<a href="https://colab.research.google.com/github/safdarjung/DL_Projects/blob/main/Text_Classification_1stop_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install tensorflow
# !pip install tensorflow-hub
# !pip install tensorflow-datasets

In [46]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# keras 3.x is showing valueerror for hub_layer, so using older versuion of kerasr

version_fn = getattr(tf.keras, "version", None)
if version_fn and version_fn().startswith("3."):
  import tf_keras as keras
else:
  keras = tf.keras

In [47]:
train_data,val_data,test_data = tfds.load(name="imdb_reviews",split = ('train','test[:40%]','test[40%:]'),as_supervised=True)

In [48]:
train_eg,label_eg = next(iter(train_data.batch(10)))

In [49]:
train_eg

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [50]:
label_eg

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [53]:

embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

hub_layer = hub.KerasLayer(embedding,input_shape=[],dtype=tf.string,trainable=True)


In [54]:

hub_layer(train_eg[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.7657859 , -3.882232  ,  3.913424  , -1.5557289 , -3.3362343 ,
        -1.7357956 , -1.9954445 ,  1.298955  ,  5.081597  , -1.1041285 ,
        -2.0503852 , -0.7267516 , -0.6567596 ,  0.24436145, -3.7208388 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489715 , -1.1315986 ],
       [ 1.8804485 , -2.5852385 ,  3.4066994 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.7855542 ,  1.3874227 ,  3.8476458 , -0.9256539 ,
        -1.896706  ,  1.2113281 ,  0.11474716,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015903 , -1.6390051 ],
       [ 0.71152216, -0.63532174,  1.7385626 , -1.1168287 , -0.54515934,
        -1.1808155 ,  0.09504453,  1.4653089 ,  0.66059506,  0.79308075,
        -2.2268343 ,  0.07446616, -1.4075902 , -0.706454  , -1.907037  ,
         1.4419788 ,  1.9551864 , -0.42660046, -2.8022065 ,  0.43727067]],
      dtype=float32)>

In [55]:
model = keras.Sequential([
  hub_layer,
  keras.layers.Dense(16, activation='relu'),
  keras.layers.Dense(1)
])

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_7 (KerasLayer)  (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400373 (1.53 MB)
Trainable params: 400373 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
print("available" if tf.config.experimental.list_physical_devices('GPU') else "not available")

available


In [42]:
model.compile(optimizer='adam',loss=keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])

In [44]:
history = model.fit(train_data.shuffle(10000).batch(100),epochs=25,validation_data=val_data.batch(100),verbose=1)

Epoch 1/25
250/250 [==============================] - 40s 148ms/step - loss: 0.5603 - accuracy: 0.6976 - val_loss: 0.4791 - val_accuracy: 0.7577
Epoch 2/25
250/250 [==============================] - 33s 132ms/step - loss: 0.4088 - accuracy: 0.8050 - val_loss: 0.3789 - val_accuracy: 0.8319
Epoch 3/25
250/250 [==============================] - 22s 88ms/step - loss: 0.3160 - accuracy: 0.8607 - val_loss: 0.3245 - val_accuracy: 0.8523
Epoch 4/25
250/250 [==============================] - 18s 70ms/step - loss: 0.2623 - accuracy: 0.8891 - val_loss: 0.3130 - val_accuracy: 0.8499
Epoch 5/25
250/250 [==============================] - 14s 55ms/step - loss: 0.2244 - accuracy: 0.9070 - val_loss: 0.3015 - val_accuracy: 0.8627
Epoch 6/25
250/250 [==============================] - 13s 52ms/step - loss: 0.1952 - accuracy: 0.9224 - val_loss: 0.2961 - val_accuracy: 0.8684
Epoch 7/25
250/250 [==============================] - 9s 37ms/step - loss: 0.1707 - accuracy: 0.9347 - val_loss: 0.2971 - val_accuracy

In [45]:
results = model.evaluate(test_data.batch(100),verbose=2)

for name,value in zip(model.metrics_names,results):
  print("%s : %.3f"%(name,value))

150/150 - 1s - loss: 0.8973 - accuracy: 0.8333 - 1s/epoch - 9ms/step
loss : 0.897
accuracy : 0.833
